In [ ]:
#Importação dos dados e arquivos da tarefa

!git clone https://github.com/GabrielPlaza95/T1-Redes-Neurais.git

%cd T1-Redes-Neurais

In [ ]:
#Importação de bibliotecas e definições auxiliares

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, f1_score, recall_score
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.backend import clear_session
from imblearn.over_sampling import SMOTE
from itertools import product as cartesian_product

def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'b--')
    plt.plot(epochs, val_metrics, 'r-')
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

def describe_model(model, model_hyper_params):
    model.summary()

    hp = model_hyper_params[model.name]
    print()
    print("Neurônios na camada escondida:", hp["neurons"])
    print("Registros por etapa:", hp["batch_size"])
    print("Função de ativação da camada escondida:", hp["activation"])
    print("Otimizador:", hp["optimizer"])

def print_results(y_pred, y_test):
    print("Resultados de Teste:\n")

    #Decodificação das saídas
    y_pred_dec = np.argmax(y_pred, axis = 1)
    y_test_dec = np.argmax(y_test, axis = 1)

    cm = confusion_matrix(y_test_dec, y_pred_dec)
    cr = classification_report(y_test_dec, y_pred_dec)
    print(cr)
    print("Matriz de Confusão:\n")
    print(np.matrix(cm))


In [ ]:
#Leitura dos dados
dataset = pd.read_csv('data/cell_train.csv') #You need to change #directory accordingly

#Visualização dos Dados
print("Amostra dos dados:")
print(dataset.head(5))

#sns.pairplot(dataset, hue="price_range")

In [ ]:
#Conversão para array
atributos = 20
X = dataset.iloc[:, :atributos].values
y = dataset.iloc[:, atributos:atributos + 1].values

#Normalização
sc = StandardScaler()
X = sc.fit_transform(X)

#Categorização one-hot da saída
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,test_size = 0.20, random_state = 19)

sm = SMOTE(random_state = 19)
X_res_train, y_res_train = sm.fit_resample(X_train, y_train)

X_train = X_res_train
y_train = y_res_train

#Hiperparâmetros a serem avaliados

num_neurons = (200, 100)
batch_sizes = (256, 128, 64)
activations = ("relu", "tanh")
optimizers = ("sgd", "adam")

hyper_params = cartesian_product(num_neurons, batch_sizes, activations, optimizers)
model_hyper_params = {}

max_val_acc = 0

for (i, (neurons, batch_size, activation, optimizer)) in enumerate(hyper_params):
    model_name = f"Modelo_{i+1}"
    model_hyper_params[model_name] = {
        "neurons": neurons,
        "batch_size": batch_size,
        "activation": activation,
        "optimizer": optimizer
    }
    model = Sequential(name = model_name)
    model.add(Dense(neurons, activation = activation, input_shape = (atributos, ), name = "Escondida"))
    model.add(Dense(4, activation = 'softmax', name = "Saida"))
    
    model.compile(
        optimizer = optimizer,
        loss = 'categorical_crossentropy', 
        metrics = ['accuracy']
    )
    
    ces = EarlyStopping(
        monitor = 'val_accuracy',
        patience = 10,
        min_delta = 0.001,
        mode = 'max',
        restore_best_weights = True,
        start_from_epoch = 50
    )

    history = model.fit(
        X_train,
        y_train,
        batch_size = batch_size,
        epochs = 500,
        validation_split = 0.2,
        verbose = 0,
        callbacks = [ces]
    )
    
    print('\n########################################################################\n')
    describe_model(model, model_hyper_params)

    plot_metric(history, 'loss')
    plot_metric(history, 'accuracy')

    #Gravando modelo de maior acurácia na validação
    val_acc = history.history['val_accuracy'][-1]
    print("Acurácia de validação:", val_acc)

    if (val_acc > max_val_acc):
        max_val_acc = val_acc
        model.save('best_model.h5')

    y_pred = model.predict(X_test, verbose = 0)
    print_results(y_pred, y_test)

In [ ]:
#Melhor modelo salvo
print('Melhor Modelo:\n')

saved_model = load_model('best_model.h5')
describe_model(saved_model, model_hyper_params)

y_pred = saved_model.predict(X_test, verbose = 0)
print_results(y_pred, y_test)

#Reset
model_hyper_params = {}
clear_session()